## Testing Critical Database Features

In [1]:
import sys
import os
sys.path.append("C:\\Users\\carlos\\Projects\\CSC400\\CSC400\\")

from sqlalchemy.orm import sessionmaker
#connect.py method that handles all connections, returns engine.
from connect import db_connect 
from models import Base, User, Post, Interaction, Media, MediaCollection
import models
from models import insert_user, get_user, exists_user, insert_interaction, \
    insert_post, exists_post, get_post, get_latest_post, get_latest_posts

#data base connections
engine = db_connect()
Session_MySQLdb = sessionmaker(engine)
db_session = Session_MySQLdb()
models.session = db_session
Base.metadata.create_all(engine)

### Creating Users

In [2]:
kepler = insert_user("kepler", "kepler@email.com", "password")
galileo = insert_user("galileo", "galileo@email.com", "password2")
bohr = insert_user("bohr", "bohr@email.com", "password3")

print("\nCreated users " + str(kepler.user_name) + ", " \
      + str(galileo.user_name) + " and " + str(bohr.user_name))

print(str(kepler.user_name) + " unique id: " + str(kepler.id) + "\n" + \
      str(galileo.user_name) + " unique id: " + str(galileo.id) + "\n" + \
      str(bohr.user_name) + " unique id: " + str(bohr.id)
     )

No User found.
Creating new user with user_name: kepler.
No User found.
Creating new user with user_name: galileo.
No User found.
Creating new user with user_name: bohr.

Created users kepler, galileo and bohr
kepler unique id: 1
galileo unique id: 2
bohr unique id: 3


### Trying to Create Existing Users

In [3]:
#if user exists, returns that user object.
kepler = insert_user("kepler", "kepler@email.com", "password")
galileo = insert_user("galileo", "galileo@email.com", "password2")
bohr = insert_user("bohr", "bohr@email.com", "password3")

User kepler already exists.
User galileo already exists.
User bohr already exists.


### Updating Users

In [4]:
kepler_bio = """Johannes Kepler was a German astronomer, mathematician, 
astrologer, natural philosopher and writer on music. He is a key figure in 
the 17th-century Scientific Revolution, best known for his laws of planetary motion, 
and his books Astronomia nova, Harmonice Mundi, and Epitome Astronomiae Copernicanae. 
These works also provided one of the foundations for Newton's theory of universal 
gravitation."""

print(f"before updating bio, bios are {kepler.bio}")

#updating fields with built in method
kepler.update(bio=kepler_bio, 
              first_name="Johannes", 
              last_name="Kepler",
              date_of_birth="12/27/1571")

print("full name: " + kepler.first_name + " " + kepler.last_name + "\n")
print("user created: " + str(kepler.date_created) + "\n" \
      "user updated: " + str(kepler.last_updated))

before updating bio, bios are NULL
setting column: bio to value: Johannes Kepler was a German astronomer, mathematician, 
astrologer, natural philosopher and writer on music. He is a key figure in 
the 17th-century Scientific Revolution, best known for his laws of planetary motion, 
and his books Astronomia nova, Harmonice Mundi, and Epitome Astronomiae Copernicanae. 
These works also provided one of the foundations for Newton's theory of universal 
gravitation.
setting column: first_name to value: Johannes
setting column: last_name to value: Kepler
setting column: date_of_birth to value: 12/27/1571
full name: Johannes Kepler

user created: 2023-03-30 03:12:36.540412
user updated: 2023-03-30 03:12:49.612940


### Posting

In [5]:
#using insert_post method, requires user object
post_kepler_1 = insert_post(kepler, "first post! just saw a supernova lol")

post_galileo_1 = insert_post(galileo, "first of all, the earth revolves around the sun.")

#using class method in the user object
post_kepler_2 = kepler.post("second post, Magini is being really annoying.")

#replying to a post
reply_galileo_1 = post_kepler_2.insert_reply(galileo, "never liked him.")

kepler.post("trying some new things lately.")

kepler.post("love my mustache!")

galileo.post("posting sometimes about the sun.")

galileo.post("this will be my last post.")

#### Getting posts from users

In [6]:
#user object contains list of all posts by that user, 
#using sqlalchemy orm relationships.
for post in kepler.posts:
    print("text: " + str(post.text) + " time: " + str(post.timestamp))
print("\n")
for post in galileo.posts:
    print("text: " + str(post.text) + " time: " + str(post.timestamp))

text: love my mustache! time: 2023-03-30 03:12:54.380137
text: trying some new things lately. time: 2023-03-30 03:12:54.373328
text: second post, Magini is being really annoying. time: 2023-03-30 03:12:54.351772
text: first post! just saw a supernova lol time: 2023-03-30 03:12:54.324733


text: posting sometimes about the sun. time: 2023-03-30 03:12:54.380137
text: this will be my last post. time: 2023-03-30 03:12:54.380137
text: first of all, the earth revolves around the sun. time: 2023-03-30 03:12:54.343358
text: never liked him. time: 2023-03-30 03:12:36.540412


In [7]:
#get_latest_posts excludes replies
posts_ = get_latest_posts(kepler, 4)

for post in posts_:
    print("text: " + str(post.text) + " time: " + str(post.timestamp))

print("\n")
posts_ = models.get_latest_posts(galileo, 4)

for post in posts_:
    print("text: " + str(post.text) + " time: " + str(post.timestamp))

text: love my mustache! time: 2023-03-30 03:12:54.380137
text: trying some new things lately. time: 2023-03-30 03:12:54.373328
text: second post, Magini is being really annoying. time: 2023-03-30 03:12:54.351772
text: first post! just saw a supernova lol time: 2023-03-30 03:12:54.324733


text: posting sometimes about the sun. time: 2023-03-30 03:12:54.380137
text: this will be my last post. time: 2023-03-30 03:12:54.380137
text: first of all, the earth revolves around the sun. time: 2023-03-30 03:12:54.343358


#### Liking Posts

In [8]:
#get latest post of a user
post_to_be_liked = get_latest_post(galileo)

print("text of post: " + str(post_to_be_liked.text))

print("galileo likes his own post.")

post_to_be_liked.like(galileo)

print("like count: " + str(post_to_be_liked.like_count))

print("bohr likes the post.")
post_to_be_liked.like(bohr)

print("like count: " + str(post_to_be_liked.like_count))

print("kepler likes the post.")
post_to_be_liked.like(kepler)

print("like count: " + str(post_to_be_liked.like_count))

text of post: posting sometimes about the sun.
galileo likes his own post.
like count: 1
bohr likes the post.
like count: 2
kepler likes the post.
like count: 3
